In [ ]:
"""
Testing MLE Recovery for out multiple CDM models
"""

In [1]:
from Utils import simple_parameters, simple_datasets, Series

In [2]:
params = simple_parameters(n=2)

In [3]:
data = simple_datasets(params, n_trials_list=[100, 200])

In [4]:
P = Series()

In [5]:
A=[0.05, 9, .05]
T=[0.05, 18, .05]

In [6]:
# Define bounds for each parameter in the model, chosen to reflect reasonable limits for the CDM model
import numpy as np
bounds = [
    (0.5, 3.0),  # decision criterion (a)
    (0.1, 3.0),  # drift rate magnitude (v)
    (0.0, 2*np.pi),  # drift angle (bias)
    (0.1, 1.0),  # non-decision time (t0)
]

In [7]:
from MLECDM import optimize_parameters_de

In [8]:
results = optimize_parameters_de(params, data, A, T, P, bounds, maxiter=1000, popsize=5)

differential_evolution step 1: f(x)= -3356.801439902625
differential_evolution step 2: f(x)= -3356.801439902625
differential_evolution step 3: f(x)= -3356.801439902625
differential_evolution step 4: f(x)= -3957.8174693233914
differential_evolution step 5: f(x)= -3957.8174693233914
differential_evolution step 6: f(x)= -3957.8174693233914
differential_evolution step 7: f(x)= -4019.5060840124906
differential_evolution step 8: f(x)= -4678.50224291543
differential_evolution step 9: f(x)= -4678.50224291543
differential_evolution step 10: f(x)= -4797.206248319971
differential_evolution step 11: f(x)= -4841.457380917328
differential_evolution step 12: f(x)= -5210.233970885955
differential_evolution step 13: f(x)= -5442.668790930266
differential_evolution step 14: f(x)= -5518.80592230995
differential_evolution step 15: f(x)= -5543.231023493617
differential_evolution step 16: f(x)= -6159.351850375643
differential_evolution step 17: f(x)= -6159.351850375643
differential_evolution step 18: f(x)= -

In [36]:
import numpy as np

def rmse(initial_params, recovered_params, param_names=None):
    """
    Calculate RMSE for each parameter separately by trial number.
    
    Parameters:
    initial_params (list of lists): Nested lists containing initial parameters for each set.
    recovered_params (dict): Dictionary of recovered parameters structured as {set_index: {trial_number: {...}}}.
    param_names (list of str, optional): List of parameter names. Used to label results.
    
    Returns:
    dict: RMSE values grouped by trial number and parameter for each set.
    """
    if param_names is None:
        param_names = [f"param_{i}" for i in range(len(initial_params[0]))]
    
    results = {set_index: {} for set_index in recovered_params.keys()}
    
    for set_index, trials in recovered_params.items():
        initial_set = initial_params[set_index]
        
        for trial_number, trial_data in trials.items():
            # Extract recovered parameters from the 'x' field
            recovered_set = trial_data.get('x', [])
            if recovered_set is None or len(recovered_set) == 0 or len(recovered_set) != len(initial_set):
                continue  # Skip if recovered parameters are missing or dimensions don't match
            
            # Initialize trial-specific entry
            if trial_number not in results[set_index]:
                results[set_index][trial_number] = {param_name: None for param_name in param_names}
            
            # Calculate squared differences and RMSE per parameter
            for i, param_name in enumerate(param_names):
                diff = (initial_set[i] - recovered_set[i]) ** 2
                results[set_index][trial_number][param_name] = np.sqrt(diff)
    
    return results

In [37]:
param_names = [
    "decision_criterion","drift_length", "drift_angle", 
    "non_decision_time"
]

In [38]:
RMSE = rmse(params, results, param_names)

In [39]:
print(RMSE)

{0: {100: {'decision_criterion': 1.7104081452957907, 'drift_length': 0.371995052443852, 'drift_angle': 0.5822521165440477, 'non_decision_time': 0.4251256565875853}, 200: {'decision_criterion': 1.7188531820241204, 'drift_length': 0.38136112674829015, 'drift_angle': 0.5822521165440477, 'non_decision_time': 0.4251256565875853}}, 1: {100: {'decision_criterion': 0.07901471546570615, 'drift_length': 0.5009943513201981, 'drift_angle': 3.2022773152953676, 'non_decision_time': 0.06789252560614756}, 200: {'decision_criterion': 0.15461563952630808, 'drift_length': 0.4860593629838471, 'drift_angle': 3.2022773152953676, 'non_decision_time': 0.13318356457373254}}}


In [15]:
from MLECDM import optimize_cdm

In [16]:
results_tqdm = optimize_cdm(params, data, A, T, P, bounds, maxiter=1000, popsize=5)

Optimizing parameter sets: 100%|██████████| 2/2 [00:48<00:00, 24.02s/it]


In [41]:
import matplotlib.pyplot as plt

def plot(initial_params, recovered_params, param_names=None):
    """
    Plot initial vs. recovered parameters for each trial number and parameter.
    
    Parameters:
    initial_params (list of lists): Nested lists containing initial parameters for each set.
    recovered_params (dict): Dictionary of recovered parameters structured as {set_index: {trial_number: {...}}}.
    param_names (list of str, optional): List of parameter names. Used to label plots.
    """
    if param_names is None:
        param_names = [f"param_{i}" for i in range(len(initial_params[0]))]
    
    for set_index, trials in recovered_params.items():
        initial_set = initial_params[set_index]
        
        for trial_number, trial_data in trials.items():
            recovered_set = trial_data.get('x', [])
            if recovered_set is None or len(recovered_set) == 0 or len(recovered_set) != len(initial_set):
                continue  # Skip if recovered parameters are missing or dimensions don't match
            
            # Plot each parameter
            for i, param_name in enumerate(param_names):
                plt.figure(figsize=(6, 6))
                plt.scatter(
                    [initial_set[i]], [recovered_set[i]],
                    color='blue', label='Recovered', s=100
                )
                plt.scatter(
                    [initial_set[i]], [initial_set[i]],
                    color='red', label='Initial', s=100, marker='x'
                )
                plt.plot(
                    [min(initial_set[i], recovered_set[i]) - 0.1, max(initial_set[i], recovered_set[i]) + 0.1],
                    [min(initial_set[i], recovered_set[i]) - 0.1, max(initial_set[i], recovered_set[i]) + 0.1],
                    'k--', label='y=x'
                )
                
                plt.title(f"Set {set_index}, Trial {trial_number}, Parameter: {param_name}")
                plt.xlabel("Initial Value")
                plt.ylabel("Recovered Value")
                plt.legend()
                plt.grid(True)
                plt.tight_layout()
                plt.show()

In [ ]:
plot(params, results, param_names)

In [ ]:
import scipy.stats as stats

def plot2(initial_params, recovered_params, param_names=None):
    """
    Plot initial vs. recovered parameters for each trial number and parameter, including correlation coefficient.
    """
    if param_names is None:
        param_names = [f"param_{i}" for i in range(len(initial_params[0]))]
    
    for set_index, trials in recovered_params.items():
        initial_set = initial_params[set_index]
        
        for trial_number, trial_data in trials.items():
            recovered_set = trial_data.get('x', [])
            if recovered_set is None or len(recovered_set) == 0 or len(recovered_set) != len(initial_set):
                continue  # Skip if recovered parameters are missing or dimensions don't match
            
            # Plot each parameter
            for i, param_name in enumerate(param_names):
                plt.figure(figsize=(6, 6))
                plt.scatter(
                    [initial_set[i]], [recovered_set[i]],
                    color='blue', label='Recovered', s=100
                )
                plt.scatter(
                    [initial_set[i]], [initial_set[i]],
                    color='red', label='Initial', s=100, marker='x'
                )
                plt.plot(
                    [min(initial_set[i], recovered_set[i]) - 0.1, max(initial_set[i], recovered_set[i]) + 0.1],
                    [min(initial_set[i], recovered_set[i]) - 0.1, max(initial_set[i], recovered_set[i]) + 0.1],
                    'k--', label='y=x'
                )
                
                # Calculate correlation
                correlation, _ = stats.pearsonr([initial_set[i]], [recovered_set[i]])
                
                plt.title(f"Set {set_index}, Trial {trial_number}, Parameter: {param_name}")
                plt.xlabel("Initial Value")
                plt.ylabel("Recovered Value")
                plt.text(0.1, 0.9, f"Corr: {correlation:.2f}", transform=plt.gca().transAxes, fontsize=12)
                plt.legend()
                plt.grid(True)
                plt.tight_layout()
                plt.show()
